In [1]:
# https://huggingface.co/docs/transformers/en/model_doc/llama3
import transformers
import torch

# https://huggingface.co/meta-llama/Llama-3.2-3B
model_id = "meta-llama/Llama-3.2-3B"

# pipeline = transformers.pipeline("text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto")
pipeline = transformers.pipeline("text-generation", model=model_id, torch_dtype=torch.bfloat16, device_map="auto")
print(pipeline.generation_config)
print(f"{pipeline.device=}")
print(f"{pipeline.model.dtype=}")

SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /meta-llama/Llama-3.2-3B/resolve/main/config.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)')))"), '(Request ID: ef0b6ebe-612f-46e2-ab58-2c88fc693c1b)')

In [2]:
pipeline.model.config.eos_token_id, pipeline.tokenizer.eos_token_id, pipeline.generation_config.eos_token_id

(128001, 128001, 128001)

In [3]:
input_text = "Simply put, the theory of relativity states that"

# need to set both to supress warning
pipeline.model.generation_config.pad_token_id = pipeline.model.config.eos_token_id
pipeline.tokenizer.pad_token_id=pipeline.model.config.eos_token_id

outputs = pipeline(input_text, batch_size=1, num_return_sequences=2, padding=True, truncation=True, max_length=64)
for output in outputs:
    print(output['generated_text'])
    print()

Simply put, the theory of relativity states that the speed of light is constant, and all objects move at a constant speed with respect to the speed of light. In other words, the speed of light is the same for all observers, regardless of their state of motion.
This theory was first proposed by Albert Einstein

Simply put, the theory of relativity states that the laws of physics are the same for all non-accelerating observers, and that the speed of light in a vacuum is the same for all non-accelerating observers, regardless of their relative velocities. The theory applies to all frames of reference and is



In [4]:
input_texts = [
  "Simply put, the theory of relativity states that",
  "The phenomenon of global warming refers to the",
]

pipeline.tokenizer.padding_side = "left"
outputs = pipeline(input_texts, batch_size=2, padding=True, truncation=True, max_length=64)
for output in outputs:
    print(output[0]['generated_text'])
    print()

Simply put, the theory of relativity states that the speed of light is the same for all observers, regardless of their state of motion. This means that the speed of light is constant, regardless of the speed of the observer. This theory has been confirmed by many experiments, including the Michelson-Morley experiment

The phenomenon of global warming refers to the phenomenon of the Earth’s temperature increasing due to the greenhouse effect. The greenhouse effect is the process by which the Earth’s temperature is regulated by the atmosphere. The atmosphere traps the heat that the sun radiates and the heat that the Earth emits. The greenhouse effect is



In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "mps"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side="left")
print(tokenizer.__class__)
print(tokenizer.__call__.__code__)
print()
for name, token in tokenizer.special_tokens_map.items():
    print(name, token, tokenizer.convert_tokens_to_ids(token), sep=",\t")
print()
print(f"{tokenizer.padding_side=}")
print(f"{tokenizer.eos_token=}")
print(f"{tokenizer.bos_token=}")

<class 'transformers.tokenization_utils_fast.PreTrainedTokenizerFast'>
<code object __call__ at 0x121a31a00, file "/Users/jyotirmaya.mahanta/projects/thelonejordan/personal/deeplearning.scratchpad/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py", line 2783>

bos_token,	<|begin_of_text|>,	128000
eos_token,	<|end_of_text|>,	128001

tokenizer.padding_side='left'
tokenizer.eos_token='<|end_of_text|>'
tokenizer.bos_token='<|begin_of_text|>'


In [7]:
tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})
# inputs = tokenizer.tokenize(prompts, return_tensors="np", truncation=True, padding=True, max_length=64)
inputs = tokenizer(input_texts, return_tensors="pt", truncation=True, padding=True, max_length=64)
# print(inputs)
token_ids = [i.tolist() for i in inputs["input_ids"]]
attention_mask = [i.tolist() for i in inputs["attention_mask"]]
print("\n".join([str(ids) for ids in token_ids]))
print("\n".join([str(ids) for ids in attention_mask]))
print()
outputs = tokenizer.batch_decode(token_ids, skip_special_tokens=False)
print("\n".join(outputs))

[128000, 61346, 2231, 11, 279, 10334, 315, 1375, 44515, 5415, 430]
[128001, 128001, 128000, 791, 25885, 315, 3728, 24808, 19813, 311, 279]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

<|begin_of_text|>Simply put, the theory of relativity states that
<|end_of_text|><|end_of_text|><|begin_of_text|>The phenomenon of global warming refers to the


In [8]:
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map=device)
print(model.config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "meta-llama/Llama-3.2-3B",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 24,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.47.1",
  "use_cache": true,
  "vocab_size": 128256
}



In [9]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm

In [10]:
print(f"{model.device=}")
print(f"{model.dtype=}")

model.device=device(type='mps', index=0)
model.dtype=torch.bfloat16


In [11]:
# https://huggingface.co/docs/transformers/en/main_classes/text_generation#transformers.GenerationConfig.from_pretrained.example
from transformers import GenerationConfig

generation_config, unused_kwargs = GenerationConfig.from_pretrained(model_id, return_unused_kwargs=True)
print(generation_config)
print(unused_kwargs)

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128001,
  "temperature": 0.6,
  "top_p": 0.9
}

{}


In [12]:
inputs = {k: v.to(device) for k, v in inputs.items()}
# print(inputs)

# need this again to suppress warning
model.generation_config.pad_token_id = tokenizer.pad_token_id

# https://huggingface.co/docs/transformers/en/main_classes/text_generation#transformers.GenerationMixin.generate
# https://github.com/huggingface/transformers/blob/a22a4378d97d06b7a1d9abad6e0086d30fdea199/src/transformers/generation/utils.py#L1914
# `**kwargs` passed to generate matching the attributes of `generation_config` will override them.
output_ids = model.generate(**inputs, generation_config=generation_config, max_length=64, temperature=0.9)
print()
outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
print("\n\n".join(outputs))


Simply put, the theory of relativity states that the speed of light is constant, regardless of the velocity of the observer.
The speed of light is 299,792,458 meters per second. From an observer at rest, the speed of light is always the same. But if we try to measure the speed

The phenomenon of global warming refers to the increase in the average temperature of the earth's surface over the past 50 years. According to the Intergovernmental Panel on Climate Change (IPCC), global warming is caused by the burning of fossil fuels, and especially the release of carbon dioxide (CO2
